# Statistical mechanics of fluids

## General exprssion of probability distribution of fluids in phase space

- The probability of a state in a classical fluid system is $f(x^N, p^N)$

$${f(x^N, p^N) = \frac{e^{-\beta H(x^N p^N)}}{Z}}$$

- Momentum and position coordinates are separated thanks to the form of kinetic and potential energy terms

$$H(x^N, p^N) = K(p^N) + U(x^N)$$

$$f(x^N, p^N) = \Phi(p^N) P(r^N)$$


- Kinetic energy part giveus us Maxwell-Botlzman distribution or the ideal gas part of the partition function $Z_p$ 
- Potential energy part gives us configruational parition function $Z_x$ evaluation of which is non-trivial and mostly can be done via simulaions:


$$Z(\beta, V, N) = Z_{p} \cdot Z_x  = \frac{1}{\lambda^3 N!} Q(\beta, V, N) $$

$$Q = \int dx^N e^{-\beta U(x^N)}$$

Pressure is related to Free energy as 

$$p= - \frac{\partial F}{\partial V} = \frac{\partial }{\partial V} log \int dx^N e^{-\beta U(x^N)}$$

- **Volume dependence of partition function is in the integration limits!** As volume grows, so does partition function. Therefore p is always positive. We can thus conclude that **in equilibrium pressure is always a positive quantity**

### Reduced configruational distribution functions

- **Hard fact:** $P(r^N)$ and Q do not factorize because of inteparticle interactions with stronger interactions implying stronger correlations in positions. 
<br><br>

- **Probability of many-body systems:** to find particle $1$ at $r_1$, $2$ and $r_2$ ...: 

$$\boxed{P(r^N) = P(r_1, r_2,...r_N)}$$

- **Marginal(ized) probability**

$$\boxed{\rho^{2/N} (r_1, r_2) =  \int dr_3... dr_N P(r_1, r_2,...r_N)}$$

- **Marginal(ized) probability for any particle 1 and 2**

$$\boxed{\rho^{2/N} (r_1, r_2) =  N(N-1)\int dr_3... dr_N P(r_1, r_2,...r_N)}$$


### Radial distribution function (RDF)

For an isotropic fluid, we have one point probability density:

$$\rho^{1/N} = N\frac{\int dr_2 ... dr_N}{\int dr_1...dr_N}  = N\frac{V^{N-2}}{V^N} =  \frac{N}{V}=\rho$$

The joint distribution for an ideal gas:

$$\rho^{2/N} = N(N-1)\frac{\int dr_3 ... dr_N}{\int dr_1...dr_N}  = N(N-1)\frac{V^{N-2}}{V^N} =  \approx \rho^2$$

- To measure the degree of spatial correlations, we introduce the Radial Distribution function (RDF):

$$\boxed{g(r_1, r_2)  = \frac{\rho^{2/N}(r_1, r_2)}{\rho^2}}$$

- RDF for isotropic fluids is due to translational invariance depends only on distance between particles:

$$\boxed{g(r) =  g(|r_2-r_1|)}$$

**Meaning of RDF**



- Since $\rho = \rho^{1/N}$, the **conditional probability density of finding a particle at r distance away from a tagged particle at the origin** is:

$$\frac{\rho^{2/N}(0,r)}{\rho} = \rho g(r)$$

- $\rho g(r)$ is then average density of particles at distance r given that a tagged particle is at the origin

### Coordination shells and structure in fluids

![](./figs/gr1_1.png)

![](./figs/gr2_2.png)

![](./figs/gr3_3.png)

![](./figs/gr4_4.png)

### Reversible work theorem and potential of mean force

$$\boxed{g(r) = e^{-\beta w(r)}}$$

$$\boxed{w(r) = - \beta^{-1} log [g(r)]}$$

- $w(r)$ Reversible work to bring two particles from infinity to distance r

- $g(r)$ Radial distribution function

$$\Big \langle -\frac{d  U(r^N)}{dr_1}  \Big \rangle_{r_1, r_2} = -\frac{\int dr_3...dr_N \frac{dU(r^N)}{dr_1} e^{-\beta U}}{\int dr_3...r_N e^{-\beta U}} =  \frac{\beta^{-1} \frac{d}{d r_1} \int dr_3...dr_N e^{-\beta U}}{\int dr_3...dr_N e^{-\beta U}} = \beta^{-1} \frac{d}{d r_1} log \int dr_3...dr_N e^{-\beta U} $$

$$\Big \langle -\frac{d  U(r^N)}{dr_1}  \Big \rangle_{r_1, r_2} = \beta^{-1} \frac{d}{d r_1} log N (N-1)\int dr_3...dr_N e^{-\beta U} = \beta^{-1} g(r_1, r_2)$$ 

### Thermodynamic properites of $g(r)$

$$\langle E \rangle = N \Big\langle \frac{p^2}{2m} \Big\rangle + \Big\langle \sum_{j>i} u(|r_i - r_j|)\Big \rangle$$

$$\boxed{E/N  = \frac{3}{2}k_B T +\frac{1}{2}\rho \int dr g(r) u(r) }$$

### Low density approximation for $g(r)$

$$w(r) =u(r) +\Delta w(r) $$

$$ g(r) = e^{-\beta u(r)} \Big (1 +O(\rho) \Big)$$

### Density expanesion and virial coefficients

$$\beta p = \rho + B_2(T) \rho^2+ O(\rho^3)$$

$$B_2(T) = -\frac{1}{2} \int dr (e^{-\beta u(r)}-1) $$